# Kmer challenge:


## Problem 1:
`There are 4096 (4^6) possible DNA hexamers (right?).
    ex. - AAAAAA, AAAAAT, AAAAAC, AAAAAG, AAAATA, ...`
`Given a list of 400 hexamers, find a new hexamer that is the most possible 'different' hexamer from that list of hexamers.`

The issue here is deciding on a metric by which to determine which of the strings of 6 bases is most 'different'. 
One approach would be to compute a distance metric between all strings, such as the Levenshtein distance, however this is a pairwise combinatorial that increases is complexity with each addition to the list. It is also possible to apply a dimension reduction approach, clustering the data on the basis of the composition into groups and then measuring distance from each group, but it then becomes difficult to discern what point to start at to then measure distance. The approach I am going to instead choose to take is an attempt to measure rareness or frequency of subunits of the hexamer as a proxy for differentness. 

Each hexamer is made up of a number of smaller kmers and so by taking the frequency of the constituent kmers we can look for rare motifs not shared by the majority of sequences. We can then further reduce the kmer we're using to compare to isolate sequences that are rarer and rarer. 

Take for example the hexamer ATCGAT:
This is composed of two trimers ATC and GAT, but if we take a sliding window approach, this is actually composed of four trimers ATC, TCG, CGA, GAT. Storing these motifs and then differentiating between hexamer sequences on the basis of the rarity of the motif, is one way to do a rather blunt analysis across all sequences in a single iteration. I'm not claimning it's the best way, but it will at least rank the sequences in how common the constituent kmers that make up the hexamer are.

### Code 1:

In [11]:
from time import time

from random import choice
from itertools import islice
from collections import defaultdict



def make_kmer_list(k, n):
    """ Returns a list of n length with DNA sequences of k bases."""
    bases = ['A', 'T', 'G', 'C']
    rtn_lst = []
    for i in range(n):
        rtn_lst.append(''.join([choice(bases) for i in range(k)]))
    return rtn_lst


def sliding_window(k, seq):
    """
    Returns all kmers of size k from a sequence seq.
    
    This gives us all the constituent kmers of size k that make up the sequences 
    """
    iterable = iter(seq)
    result = tuple(islice(iterable, k))
    if len(result) == k:
        yield result
    for kmer in iterable:
        result = result[1:] + (kmer,)
        yield result


def record_kmers(lst_of_seqs, k):
    """ 
    This function iterates over each of the DNA sequences and  produces all possible kmers and then records the 
    frequency of these kmers. In a seperate dictionary for each kmer we record every sequence under the key 
    of its corresponding kmer, so that we can examine these sequences later. 
    By design each sequence should be stored under multiple kmers.
    """
    codon_dct = defaultdict(list)
    codon_fequencies = defaultdict(int)

    for i in range(len(lst_of_seqs)):
        seq = lst_of_seqs[i]

        codons = sliding_window(k, seq)

        for codon in codons:
            codon = ''.join(codon)  # kmers come back as a list of characters so we'll put them back together
            codon_dct[codon].append(seq)  # add seqeuence to a list beneath the kmer 
            codon_fequencies[codon] += 1  # Incredment kmer frquency for each time it is 'seen'

    return codon_dct, codon_fequencies


def sort_dict_by_values(d, reversed):
    return sorted(d, key=d.get, reverse=reversed)


def get_target_sequences(codon_dct, sorted_codons):
    """
    Return the rarest sequences as discerned by kmer frequency.
    
    Any sequence that contains two rare kmers can be included in the targets.
    """
    
    # Now we need to get a subset of targets on which to perform further analysis so:
    kmer_not_found = True  # set a logic gate
    i = 0  # value to keep track of our position
    current_pool = codon_dct[sorted_codons[1]]  # we start with sequences containing the rarest codons as these
    # are the most likely to be the most 'different'

    # This is where we'll store our potential targets:
    targets = []

    # Iterate through until we find a target that contains 2 of the rarest codons.
    while kmer_not_found:
        i += 1
        current_sequences = codon_dct[sorted_codons[i]]
        for seq in current_sequences:
            if seq in current_pool:
                # if we find a sequence that has both the rarest and another rare kmer
                # By design this will not stop iterations because if there are ten equally rare sequences then we need
                # to differentiate between these.
                kmer_not_found = False
                targets.append(seq)
            else:
                current_pool.extend(current_sequences)           

    return targets


def reduce_by_rarity(lst_of_sequences, k):
    """ 
    For a given list of sequences, and a given size of k, we first differentiate the 'rarist' sequences
    on the basis of kmers of size k and then if there is more than one returning result we decrement k by 1 reducing 
    the kmer size to further differentiate between this pool of now smaller atrgets until we either run out of 'k' 
    e.g. k = 1 and we are differntiating on the basis of single characters or there is only one target.
    """
    sequences = lst_of_sequences
    for i in reversed(range(1, k+1)):
        # Record kmers present in each sequence, using a sliding window technique
        # To minimise iterations we record the sequences containing the kmers and the kmer frequencies separately
        kmer_dct, kmer_frequencies = record_kmers(sequences, i)

        # Sort the kmers by their frequency so we evaluate the rarest codons first
        sorted_kmers = sort_dict_by_values(kmer_frequencies, False)
        
        # Use the least frequent kmers as a search pattern to select the most different sequences
        targets = get_target_sequences(kmer_dct, sorted_kmers)
        
        # If the list of sequences is one return the target, otherwise set the targets as the sequences, lower the 
        # value of k and begin again
        if len(targets) == 1:
            return targets
        else:
            sequences = targets
    return targets


def time_function_runtime(func, *args):
    """
    Decorator function to allow me to keep track of run time.
    Note this is a only a rough estimate and I'd use the timeit library for proper timing optimisation, as a 
    single run value is essentially only one data point and may be biased in one direction or another.
    """
    def wrapper(*args):
        t = time()  # Get time at start
        func(*args)  # Run program
        t1 = time()  # Get time at completion
        print(t1-t)  # Print different between start and finish (run time)
    return wrapper


@time_function_runtime
def run(no_of_values, kmer_size, starting_window_size):
    """ A function allowing me to specify the size of the list, the size of the starting values e.g. in this challenge
    hexamers (kmer_size = 6), and the size of the sliding window to begin."""
    six_mers = make_kmer_list(kmer_size, no_of_values)  # Generate the kmers to iterate over
    six_mers = list(set(six_mers)) # Make sure we don't have duplicates
    targets = reduce_by_rarity(six_mers, starting_window_size)  # start with a sliding window of 3
    print(len(targets))
    return targets


# I chose a starting window size of 4 as it should optimise the time slightly more by getting rid of the 
# really common sequences earlier when we're iterating over less subunits.
run(40, 6, 4)
run(400, 6, 4)
run(4000, 6, 4)

1
0.0017080307006835938
1
0.0059299468994140625
1
0.03848695755004883


## Problem 2:
`Suppose that we are concerned with DNA duplex hexamers, find also the most possible 'different' duplex hexamer?
    duplex ex.- AAAAAA     TTTTTT
                ||||||  =  ||||||
                TTTTTT     AAAAAA`
                
One method to solve this would simply be to remove duplicated strings from the intial search space and keep only sequences where their complimentary sequence isn't present.

This is obviously still assuming the same inputs e.g. AAAAAA and TTTTTT as opposed to the representation above, but this could likely be handled by finding a string way of expressing the above if requires e.g. AAAAAATTTTTT= although I don't see any benefit in this and it would probably influence the kmer search space.

## Code 2:

In [13]:
def create_compliment(sequence):
    """ Function to convert between an input sequence and it's complimentary sequence."""
    base_mappings = {'A':'T','T':'A','C':'G','G':'C'}
    return ''.join([base_mappings[base] for base in list(sequence)])

def remove_complimentary_sequence(lst_of_sequences):
    """
    Iterate over the list of sequences and only keep those where the sequence and its complimentary sequence are
    not present.
    """
    rtn_lst = []
    for seq in lst_of_sequences:
        if seq not in rtn_lst and create_compliment(seq) not in rtn_lst:
            rtn_lst.append(seq)
    return rtn_lst

# Now modify run() to use the new code
@time_function_runtime
def run(no_of_values, kmer_size, starting_window_size):
    """ A function allowing me to specify the size of the list, the size of the starting values e.g. in this challenge
    hexamers (kmer_size = 6), and the size of the sliding window to begin."""
    six_mers = make_kmer_list(kmer_size, no_of_values)  # Generate the kmers to iterate over
    six_mers = list(set(six_mers)) # Make sure we don't have duplicates
    six_mers = remove_complimentary_sequence(six_mers)
    targets = reduce_by_rarity(six_mers, starting_window_size)  # start with a sliding window of 3
    print(len(targets))
    return targets

run(40, 6, 4)
run(400, 6, 4)
run(4000, 6, 4)

1
0.0008950233459472656
1
0.008295774459838867
1
0.1419692039489746


# ANALYSIS:
`Can your code work for longer k-mers? At what k-mer length does your code become a processing nightmare`
The constant reduction of the search space by starting at a greater sliding window size, that is a larger kmer, reduces the issue of a growing sequence length, but by no means gets rid of it. Also currently it decrements by 1, which is massively inefficent at large kmer sizes e.g. imagine the difference in kmer subunits for a kmer of size 2000 with subunits of size 500 and 499. It's probably not going to be an enlightening difference.

The examples below show the run time of the program for sequences of length 200 and the again for sequences of length 2000. 

Already it begins to slow, and an order of magnitude more would put the process into the minutes.

It somewhat depends on where the process is running and how long is too long, but by length 200,000 bases per sequence this approach would almost certainly become extremely tedious and redudant.

As alluded to above part of this is the issue of decrementing by 1 for each new target when the subunit size is larger than 20, i.e. we're looking at kmers of size 100 within the kmer of size 2000. Greater distance between the kmer subunits during the analysis probably inprove the speed of the process by decrementing by ks of greater size such as ten or twenty. This woukld reduce the number of interations, although this would take optimisation, to ensure we weren't throwing away information.

In conclusion the current system would work OK upto kmers of 2000 for smallish lists e.g. 4000 or less taking 40 seconds to return a target.


In [19]:
run(40,200, 20)
run(400, 200, 20)
run(4000, 200, 20)

run(40,2000, 100)
run(400, 2000, 100)
# run(4000, 2000, 100) - this takes 40 seconds so I've commented it out in case you run it.

# run(4000, 2000, 500) - this takes 70 seconds, becase you're iterating 500 times. 
# This could be fixed by decrementing by 10 or 20 instead of 1 for kmers above 20.

1
0.030164003372192383
1
0.20389413833618164
1
2.5957210063934326
1
0.30382704734802246
1
3.416584014892578
1
70.41143441200256


# DISCUSSION ON EXTENSION FEASIBILITY/STRATEGY:
`Suppose we wanted to find the most possible 'different' 20-mer in an entire genome (so the list is now all 20-mers in a whole genome!) - perhaps that's not even realistically possible...
What (if any) are some strategies that you might employ in order to find the most possible 'different' (or at least a very 'different') 20-mer?`

This depends entirely on how your classifying difference, if you want to perform a distance calculation the pairwise combination of 20-mer in a 3B base pairs looks instantly unfeasible, as would be using the kmer anaylsis outlined above as it would take forever to finish. If you simply wanted to ask have I seen this kmer or part of this kmer before, then a bloom filter approach may lend itself to this task. However, this doesn't really tell you difference as much as distinctness.

If you were confident you could homogonise the genome, that is mix up the location of these 20-mer reads sufficiently, you might be able to break the problem down in a map-reduce style manner performing an analysis on many sub-sections of the dataset in parallel. This does open itself up to the sub-population problem though, in that you are not longer comparing everything equally and sub-population structure might then bias the final result. 

Trying to extend my kmer approach, it is I suppose possible to perform this analysis if you chunked up the genome and mixed up the sequences in between each change in the sliding window size, essentially performing the rarity selection in microcosm many times. However, this risks population structure returning you a biased answer where you end up with a rare, but not the rarist sequence. 

Alternatively, if what you were interested in was difference from the norm, rather than difference from everything else then it may be possible to produce a representation of that norm that you can then compare against all the differnt 20-mers individually, allowing enormous parallelisation. At this point, it may then lend itself to string distance calculations as you would be reducing the number of pairwise combinations, although it seems unlikely that you would sufficently represent the norm with a small enough pool of sequences that this pairwise problem wouldn't still be extremely computationally intensive. 

Using this differentiation from the norm with my approach, assuming we could represent the 'norm' well enough, it would be feasible to characterise all kmers within this norm and then simply perform a kmer analysis for every new sequence looking if the kmers that compose the sequences are sufficiently different to be interesting. This may however simply return you kmers that result from sequencing or PCR error and thus are never seen in nature.



